In [1]:
import torch
from transformers import SegformerFeatureExtractor
import sys
sys.path.insert(1, '../utils/')
import os

/home/emir/anaconda3/envs/emirenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from create_dataloaders import EddyDatasetTrain, EddyDatasetValid

In [3]:
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
feature_extractor.reduce_labels = False
feature_extractor.size = 128

/home/emir/anaconda3/envs/emirenv/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [4]:
input_image_dir = "/home/emir/dev/segmentation_eddies/downloads/data4test/aug_data/"
mask_image_dir = "/home/emir/dev/segmentation_eddies/downloads/data4test/aug_label/"

In [5]:
train_len = len(os.listdir(input_image_dir))
split = int(0.85 * train_len)

In [6]:
train_data = EddyDatasetTrain(feature_extractor=feature_extractor, input_image_dir=input_image_dir, mask_image_dir=mask_image_dir, split=split)

['4_2_101_51_aug.mat', '6_19_51_101_aug.mat', '3_46_151_51_aug.mat', '3_30_151_201_aug.mat', '6_2_1_101_aug.mat', '4_43_101_1_aug.mat', '6_12_51_201_aug.mat', '7_26_1_51_aug.mat', '6_36_101_51_aug.mat', '7_2_201_51_aug.mat', '4_57_201_51_aug.mat', '7_21_244_101_aug.mat', '6_41_201_51_aug.mat', '3_26_151_1_aug.mat', '6_26_201_244_aug.mat', '5_54_151_151_aug.mat', '5_51_51_151_aug.mat', '5_44_101_151_aug.mat', '7_58_51_101_aug.mat', '3_34_151_51_aug.mat', '4_2_201_151_aug.mat', '3_16_201_101_aug.mat', '4_32_244_101_aug.mat', '3_35_1_1_aug.mat', '5_30_101_151_aug.mat', '6_31_101_201_aug.mat', '5_15_151_1_aug.mat', '4_21_201_1_aug.mat', '4_39_1_201_aug.mat', '4_51_244_244_aug.mat', '4_27_244_101_aug.mat', '4_35_1_51_aug.mat', '6_36_151_244_aug.mat', '7_38_101_244_aug.mat', '7_26_1_201_aug.mat', '3_8_101_244_aug.mat', '3_37_151_201_aug.mat', '5_37_1_201_aug.mat', '5_44_151_1_aug.mat', '5_30_1_244_aug.mat', '6_26_101_244_aug.mat', '3_23_51_1_aug.mat', '5_10_151_1_aug.mat', '3_35_201_244_aug.